In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

aapl = yf.download('AAPL', start='2010-01-01', end='2018-12-31')
adjusted_close_prices = aapl['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [2]:
BB_window = 20
rolling_mean = adjusted_close_prices.rolling(window=BB_window).mean()
rolling_std = adjusted_close_prices.rolling(window=BB_window).std()
upper_band = rolling_mean + rolling_std
lower_band = rolling_mean - rolling_std
df = pd.DataFrame(np.array([adjusted_close_prices, upper_band, lower_band]).T, columns=['price', 'upper', 'lower'], index=adjusted_close_prices.index)
df['next_5_ret'] = (df['price'] / df['price'].shift(5) - 1).shift(-5)
df.reset_index(inplace=True)
df.Date = df.Date.dt.strftime('%Y%m%d')

In [6]:
#df.dropna(axis=0, inplace=True)
df['label_1'] = np.nan
df['label_2'] = np.nan
for i in range(BB_window, len(df)):
    if df.loc[i, 'price'] < df.loc[i, 'lower']:
        df.loc[i, 'label_1'] = 'buy'
    if df.loc[i, 'price'] > df.loc[i, 'upper']:
        df.loc[i, 'label_1'] = 'sell'

for i in range(len(df)-5):
    if df.loc[i, 'next_5_ret'] < -0.02:
        df.loc[i, 'label_2'] = 'sell'
    if df.loc[i, 'next_5_ret'] > 0.02:
        df.loc[i, 'label_2'] = 'buy'
# + return
df

C:\Users\hefzh\AppData\Local\Temp\ipykernel_31852\143346381.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'buy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'label_1'] = 'buy'
C:\Users\hefzh\AppData\Local\Temp\ipykernel_31852\143346381.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'sell' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'label_2'] = 'sell'


,Date,price,upper,lower,next_5_ret,label_1,label_2
0,20100104,6.461975,NaN,NaN,-0.018223,NaN,NaN
1,20100105,6.473149,NaN,NaN,-0.031067,NaN,sell
2,20100106,6.370185,NaN,NaN,-0.001517,NaN,NaN
3,20100107,6.358407,NaN,NaN,-0.005461,NaN,NaN
4,20100108,6.400681,NaN,NaN,-0.028541,NaN,sell
...,...,...,...,...,...,...,...
2258,20181221,36.073051,42.764922,38.776021,NaN,buy,NaN
2259,20181224,35.139706,42.819024,38.112606,NaN,buy,NaN
2260,20181226,37.614292,42.670986,37.843026,NaN,buy,NaN
2261,20181227,37.370182,42.511788,37.569289,NaN,buy,NaN


In [21]:
df1 = df[['Date', 'price', 'upper', 'lower', 'label_1']].copy()
df1.dropna(axis=0, inplace=True)

df2 = df[['Date', 'price', 'upper', 'lower', 'next_5_ret', 'label_2']].copy()
df2.dropna(axis=0, inplace=True)

In [30]:
fig_window_size_list = [20]
fig_width = 0.6
fig_height = 0.64
dpi = 100

for window_size in fig_window_size_list:
    dir_path = f'window_{window_size}_plots'
    os.makedirs(dir_path, exist_ok=True)
    for i in range(window_size, df.shape[0]):
        label1 = df['label_1'].iloc[i]
        label2 = df['label_2'].iloc[i]
        next_5_ret = df['next_5_ret'].iloc[i]
        df_window = df.iloc[i-window_size: i].copy()
        start = df_window.Date.iloc[0]
        end = df_window.Date.iloc[-1]
        plot_path = os.path.join(dir_path, f"window{window_size}_{start}_{end}_{label1}_{label2}_{next_5_ret:.6f}.png")
        plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
        plt.plot(df_window['price'], color='blue')
        plt.plot(df_window['upper'], color='red')
        plt.plot(df_window['lower'], color='green')
        plt.xticks([])
        plt.yticks([])
        plt.savefig(plot_path)
        plt.close()

In [ ]:
def SMA(prices, window=20):
    sma = prices.rolling(window=window).mean()
    return sma

def Bollinger_Bands(prices, std_num=2, window=20):
    sma = SMA(prices, window=window)
    roll_std = prices.rolling(window=window).std()
    upper = sma + std_num * roll_std
    lower = sma - std_num * roll_std
    bb_value = (prices - sma) / (2 * roll_std)
    return upper, lower, bb_value

def Momentum(prices, window=20):
    momentum = prices / prices.shift(window) - 1
    return momentum

def MACD(prices):
    EMA12 = prices.ewm(span=12, adjust=False).mean()
    EMA26 = prices.ewm(span=26, adjust=False).mean()
    macd = EMA12 - EMA26
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def Oscillator(prices, stock_window=14, K_window=3):
    high_stock = prices.rolling(stock_window).max()
    low_stock = prices.rolling(stock_window).min()
    K = (prices - low_stock) / (high_stock - low_stock) * 100
    D = K.rolling(K_window).mean()
    return K, D